## Iterative Prompt Development

- We'll iteratively analyze and refine our prompts to generate marketing copy from a product fact sheet.

#### Setup

- Load the API key and relevant Python libaries.

In [12]:
from langchain_groq import ChatGroq
import langchain_core
import os
import warnings
warnings.filterwarnings('ignore')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

groq_api_key = os.getenv('GROQ_API_KEY')

In [14]:
# model='llama-3.3-70b-versatile'

llm = ChatGroq(
    api_key=groq_api_key,
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=None
)

In [16]:
def get_completion(prompt, model="llama-3.3-70b-versatile"):
    llm.model_name = model
    return llm.invoke(prompt).content

## Generate a Marketing Product Description from a Product Fact Sheet

In [18]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [20]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


**Introducing the Timeless Mid-Century Office Chair**

Elevate your home or business space with our stunning mid-century inspired office chair, part of a beautifully crafted family of furniture pieces. This chair is designed to make a statement, with a range of shell color and base finish options to suit your unique style. Choose from 10 vibrant fabric and 6 luxurious leather upholstery options, or opt for a sleek plastic back and front design.

The chair's sturdy 5-wheel aluminum base, coated with a durable plastic finish, provides smooth movement and stability. The pneumatic chair adjust feature allows for effortless height adjustments, ensuring you can work in comfort all day long. Plus, with the option to add armrests, you can customize your chair to fit your needs.

**Key Features:**

* Durable cast aluminum base with modified nylon coating
* High-density foam seat for optimal comfort
* Available with or without armrests, and with soft or hard-floor caster options
* Two seat foam 

## Issue 1 : The text is too long 

- Limit the number of words/sentences/characters.

In [29]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 100 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


Elevate your workspace with our mid-century inspired office chair, available in various shell colors and base finishes. Choose from 10 fabric and 6 leather options, with or without armrests. Durable and adjustable, it's perfect for home or business use.


In [31]:
len(response.split())

39

## Issue 2 : Text focuses on the wrong details

- Ask it to focus on the aspects that are relevant to the intended audience.

In [35]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Cast aluminum base with nylon coating and HD36 foam seat.


In [37]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Cast aluminum base, nylon PA6/PA66 coating. SWC-100 SWC-110


## Issue 3 : Description needs a table of dimensions

- Ask it to extract information and organize it in a table.

In [40]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

<div>
The product is constructed from high-quality materials, including a 5-wheel plastic coated aluminum base and a shell made from cast aluminum with a modified nylon PA6/PA66 coating. The seat is made from HD36 foam, available in two densities: medium (1.8 lb/ft3) or high (2.8 lb/ft3). The product is available with or without armrests, and the armrests come in 8 position PU options. The base finish options include stainless steel, matte black, gloss white, or chrome. The product IDs for this item are SWC-100 and SWC-110.
</div>

<table>
<caption>Product Dimensions</caption>
<tr>
<th>Dimension</th>
<th>Measurement (in)</th>
</tr>
<tr>
<td>Width</td>
<td>20.87</td>
</tr>
<tr>
<td>Depth</td>
<td>20.08</td>
</tr>
<tr>
<td>Height</td>
<td>31.50</td>
</tr>
<tr>
<td>Seat Height</td>
<td>17.32</td>
</tr>
<tr>
<td>Seat Depth</td>
<td>16.14</td>
</tr>
</table>


## Load Python libraries to view HTML

In [43]:
from IPython.display import display, HTML

In [45]:
display(HTML(response))

Dimension,Measurement (in)
Width,20.87
Depth,20.08
Height,31.50
Seat Height,17.32
Seat Depth,16.14
